In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import analyze_covid
import airport_data_explore

/home/aytang/sfuhome/353project/airport_data_explore.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  canada['Date'] = pd.to_datetime(canada['Date'])


In [3]:
covidNA = pd.read_csv('casesNA.csv',compression = 'gzip')
#traffic = pd.read_csv("airport_traffic_NA.csv")
#coviddata/casesCanada.csv, casesUS.csv

In [72]:
# Unload stored dataframes from other notebooks within directory
%store -r

# Airport Grouping
Adaptation of http://eyana.me/nearest-distance-python/

In [79]:
#airportLocations = rankNA.drop(columns = ["Version", "PercentOfBaseline"])
naAirports = na.drop(columns = ["AggregationMethod", "Version", "Centroid", "ISO_3166_2", "Geography", "geometry", "index"])
airportLocations = naAirports.drop(columns = ["Date", "PercentOfBaseline"]).drop_duplicates().dropna().reset_index(drop = True)
airportLocationsCA = airportLocations[airportLocations["Country"] == "Canada"]
airportLocationsUS = airportLocations[airportLocations["Country"] != "Canada"]
airportLocations

,AirportName,City,State,Country,lat,long
0,Los Angeles International,Los Angeles,California,United States of America (the),33.941369,-118.404993
1,San Francisco International,South San Francisco,California,United States of America (the),37.621188,-122.383935
2,Denver International,Denver,Colorado,United States of America (the),39.864347,-104.700316
3,Miami International,Miami Springs,Florida,United States of America (the),25.795724,-80.288715
4,Hartsfield-Jackson Atlanta International,College Park,Georgia,United States of America (the),33.641076,-84.427919
5,Daniel K. Inouye International,Urban Honolulu,Hawaii,United States of America (the),21.325965,-157.918285
6,Chicago OHare International,Chicago,Illinois,United States of America (the),41.980460,-87.910595
7,Boston Logan International,Boston,Massachusetts,United States of America (the),42.363633,-71.010291
8,Detroit Metropolitan Wayne County,Romulus,Michigan,United States of America (the),42.212973,-83.353731
9,Charlotte Douglas International,Charlotte,North Carolina,United States of America (the),35.213689,-80.947811


In [70]:
caLocations = dfCA[['Country/Region', "Province/State", "Latitude", "Longitude"]].drop_duplicates().dropna()
caLocations = caLocations[caLocations["Province/State"] != "Recovered"].reset_index(drop = True)
caLocations

,Country/Region,Province/State,Latitude,Longitude
0,Canada,Ontario,51.2538,-85.3232
1,Canada,British Columbia,53.7267,-127.6476
2,Canada,Alberta,53.9333,-116.5765
3,Canada,Quebec,52.9399,-73.5491
4,Canada,Saskatchewan,52.9399,-106.4509
5,Canada,Nova Scotia,44.6820,-63.7443
6,Canada,Manitoba,53.7609,-98.8139
7,Canada,New Brunswick,46.5653,-66.4619
8,Canada,Newfoundland and Labrador,53.1355,-57.6604
9,Canada,Prince Edward Island,46.5107,-63.4168


### Groups for Canadian Airports
*Did this manually. 

Some provinces don't have data for their airports. Figure out what to do.
Assume that no Canadian travelled to US airports
I used www.travelmath.com/nearest-airport to make guesses

ON - Toronto Pearson, Hamilton International	
BC - Vancouver International	
AB - Calgary International, Edmonton International	
QC - Montreal Mirabel, Montreal Trudeau	
SK - ?
NS - Halifax International	
MB - Winnipeg International
NB - (Halifax International?)
NFL - ?
PEI - (Halifax International?)
NT - (Edmonton International?)
YK - (Vancouver International?)

#### TODO: Will repeat again with script (airport_grouping.ipynb)

In [73]:
usLocations = dfUS[['Country/Region', "County", "Province/State", "Latitude", "Longitude"]].drop_duplicates().dropna()
usLocations = usLocations[usLocations['Latitude'] != 0].reset_index(drop = True)
usLocations

,Country/Region,County,Province/State,Latitude,Longitude
0,US,New York City,New York,40.767273,-73.971526
1,US,Nassau,New York,40.740665,-73.589419
2,US,Westchester,New York,41.162784,-73.757417
3,US,Suffolk,New York,40.883201,-72.801217
4,US,Rockland,New York,41.150279,-74.025605
5,US,Orange,New York,41.403375,-74.302408
6,US,Albany,New York,42.600603,-73.977239
7,US,Dutchess,New York,41.764861,-73.743567
8,US,Monroe,New York,43.146389,-77.693229
9,US,Erie,New York,42.762490,-78.730637


In [78]:
%store usLocations
%store caLocations
%store airportLocationsUS
%store airportLocationsCA
%store airportLocations

Stored 'usLocations' (DataFrame)
Stored 'caLocations' (DataFrame)
Stored 'airportLocationsUS' (DataFrame)
Stored 'airportLocationsCA' (DataFrame)
Stored 'airportLocations' (DataFrame)
